# Загружаем необходимые пакеты и импортируем библиотеки

---

In [1]:
# устанавливаем библиотеку implicit 
%pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 21.7 MB/s eta 0:00:00


In [2]:
# загружаем библиотеки
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
import implicit
import tensorflow as tf
from sklearn import preprocessing

In [5]:
# Проверяем что у нас работает GPU
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
#GPU номер и имя
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-0b83122c-d41f-db7c-2901-46531475ad9d)


# Подготовка и обработка данных

---

In [7]:
# загружаем данные из источника
data = pd.read_parquet('train_mfti.parquet', engine='pyarrow')

In [138]:
# создаем копию - на тестах было удобно для откатов
df = data.copy()

In [139]:
# посмотрим на данные 
df.head()

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy


In [140]:
# и так тоже посмотрим
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292588 entries, 0 to 12292587
Data columns (total 6 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   event_date       object
 1   event_timestamp  int64 
 2   vacancy_id_      int64 
 3   cookie_id        object
 4   user_id          object
 5   event_type       object
dtypes: int64(2), object(4)
memory usage: 562.7+ MB


In [141]:
# Дубликатов нет в данных - но на всякий случай запустим удаление
df = df.drop_duplicates()

In [142]:
# Мы знаем где пропуски, но смотреть это - добрая традиция и это никогда не лишняя строчка
display(df.isnull().sum())

event_date               0
event_timestamp          0
vacancy_id_              0
cookie_id                0
user_id            3556099
event_type               0
dtype: int64

---
Очевидно, что пропуски в колонке с зарегистрированными пользователями, то есть более 3 миллионов взаимодействий у нас прошли по незарегистрированным пользователям, но для нас эти пропуски не страшны, потому что:

---

In [143]:
# удаляем колонки, которые не будут принимать участия в обучении
df.drop(columns=['event_date', 'event_timestamp', 'user_id'], axis=1, inplace=True)

---
Да, мы решили удалить данные о пользователях и учить модель только на cookie_id, игнорируя user_id. Мы реализовали код который связывает cookie_id c user_id. Но к нему есть несколько вопросов, он ориентирван на ситуацию когда один пользователь использует несколько устройств для работы на сайте, но есть и обратная схема - есть устройства, которые использовались несколькими пользователями, в одном слечае на одном устройстве заходили на сайт 57 пользователей! Такую обратную связь мы отразить в коде не сможем потому по условию задачи, потому что по формату итогового задания нам на вход приходят только cookie_id, без указания пользователя и понять кто это из тех 57, а может и новый пользователь, мы не сможем. Повторюсь мы реализовали алгоритм, который хорошо решает задачу соединения пользователя или устройства если человек пользуется скажем двумя гаджетами - программа соотнесет их с 1 конкретным пользователем, для незарегистрированных пользователей алгоритм будет учится на номере устройства, а для таких ситуаций, где 57 человек на один компьютер, код запоминает последнего пользователя (может это и логично). С этим кодом можно ознакомиться в ноутбуке implict_build_v4 в папке IN DEVELOPMENT. Почему его нет в финальном ответе - все банально при такой реализации пусть немного, но ухудшаеся метрика, поэтому мы решили работать cookie_id. Возможно если углубиться в анализ и добавить психологии, то мы увидим что человек на разных устройствах реализует разные траектории поиска,как разные варианты развития. Но пока банально метрика.

---

In [144]:
# для удобства восприятия немного выравняем данные
df = df[['cookie_id', 'vacancy_id_', 'event_type']]

In [145]:
df.head()

,cookie_id,vacancy_id_,event_type
0,97990f1a021d4be19aa3f955b7eacab4,129850,show_vacancy
1,03bf8c511fa949c79845a5d81b09aa1d,108347,show_vacancy
2,03bf8c511fa949c79845a5d81b09aa1d,109069,show_vacancy
3,03bf8c511fa949c79845a5d81b09aa1d,171425,show_vacancy
4,03bf8c511fa949c79845a5d81b09aa1d,252384,show_vacancy


 ### Обработка данных первая исследовательская позиция (Маштабирование взаимодействий и их силы)
 ---

In [146]:
# Вот тут мы посмотрим на количество определенного вида взаимодействий в данных
df['event_type'].value_counts()

show_vacancy              6180832
preview_click_vacancy     4758461
click_response             382828
click_contacts             276819
preview_click_response     190130
click_favorite             155472
preview_click_favorite     106622
preview_click_contacts     101231
click_phone                 78667
preview_click_phone         15927
Name: event_type, dtype: int64

* 1 show_vacancy - просмотр вакансии
* 2 preview_click_vacancy - клик по карточке вакансии
* **3 click_response - отклик со страницы вакансии**
* **4 preview_click_response - отклик с карточки вакансии**
* 5 click_favorite - добавление вакансии в избранное  со страницы вакансии
* 6 preview_click_favorite - добавление вакансии в избраное с карточки вакансии
* **7 click_contacts - клик на контакты со страницы вакансии**
* **8 preview_click_contacts - клик на контакты из карточки вакансии**
* **9 click_phone - клик на номер телефона, указанный в вакансии**
* **10 preview_click_phone - клик на номер телефона из карточки вакансии**

Жирным выделены искомые ""откликнется"" и ""позвонит"""
! Стоит обратить внимание на то, что позиции 3 и 4 по частоте идут выше чем позиции 5 и 6, которые нам в положительное взаимодействие не засчитываются.

---
Наши испытания показали, что неплохой результат метрика показывает, при маштабировании взаимодействий от 1 до 10 сообразно уменьшению их частоты, в данных. Но тут мы видим что 2 отрицательных типа взаимодействий:

* click_favorite             155472
* preview_click_favorite     106622 

получают вес выше чем:

* click_response             382828
* click_contacts             276819
* preview_click_response     190130

---

In [332]:
# Вот это разбиение по нашему заданию показало наилучшие результаты (отричательные взаимодействия все одинкавые, а положительные имеют вес)
df['eventStrength'] = df['event_type'].map({'show_vacancy': 1.0,
                                     'preview_click_vacancy': 1.0,
                                     'click_favorite': 1.0,
                                     'preview_click_favorite': 1.0,
                                     'click_response': 5.0,
                                     'click_contacts': 6.0,
                                     'preview_click_response': 7.0,
                                     'preview_click_contacts': 8.0,
                                     'click_phone': 9.0,
                                     'preview_click_phone': 10.0})

In [333]:
# проверим как мы закодировали взаимодействия
df['eventStrength'].value_counts()

1.0     11201387
5.0       382828
6.0       276819
7.0       190130
8.0       101231
9.0        78667
10.0       15927
Name: eventStrength, dtype: int64

Группировка данных.

---
Еще один point по которому проводилось исследование. Группировка схлопывает взаимодействия и выдает результат в виде силы взаимодействия. Пробовались многие варианты: сумма, максимум, среднее. Пробовался вариант когда мы выделяли только уникальные завимодействия для каждой пары cookie_id и vaconcy_id_ и уже потом применяли агрегацию и маштабирование (с кодом можно ознакомиться в ноутбуке implict_build_v4). Но метрика показывает лучшие результаты именно с суммой по всем свзаимодействиям. Этому есть математическое и логическое объяснение. Работа сайта с учетом, рекламных компаний приоретизации предложений от партнеров и т.д., устроена так, что некоторые вакансии (например вакансии недели) показываются пользователю несколко раз (мы это протестировали), а значит шанс того что пользователь положительно повзаимодействует с вакансией недели увеличивается существенно. То есть чем больше показали, тем больше вероятность клика. Этот тезис подтвержается нашим baseline алгоритмом в котором из различных вариантов ТОП вакансий в числе первых по метрике ТОП построенный по количеству взаимодействий.

---

In [334]:
# сгруппируем сочетания ваканции и человека при этом суммируем историю из взаимодействия в числах, чем больще число
# тем больше интерес человека к вакансии - а значит больше и сила взаимодействия

grouped_df = df.groupby(['cookie_id', 'vacancy_id_']).sum('numeric_only').reset_index()

In [335]:
# посмотрим на то что получилось
grouped_df.sample(10)

,cookie_id,vacancy_id_,eventStrength
2326934,7eaec46c02f4426088087144512cdef1,226854,2.0
3872045,d38ba66a917a48b0a55a088ed81dfb3b,243569,2.0
1014878,373546f9ac9142dd934c6231426362db,213418,2.0
3446597,bc21534101974291aa6f4d26b206019a,110983,2.0
3563911,c299092cb4604fb69b6b75ca2adf3dfb,205335,16.0
418999,16f521977c5844058b3b1cd9c8f8d945,151737,2.0
1844511,64a467946694402a93cbb43b5c7afbcd,147623,2.0
2230608,79964be693754647a01263872959a195,260161,1.0
3242605,b0efd29b8c66466482355c862c355351,161790,7.0
935488,32e6841d89cd4151a7ebf7ec7477936b,182100,2.0


In [336]:
# с 12 000 000 + наши данные схлопнулись до 4 500 000 +
grouped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4678585 entries, 0 to 4678584
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   cookie_id      object 
 1   vacancy_id_    int64  
 2   eventStrength  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.1+ MB


In [337]:
# посмотрим на одну вакансию в ключе ее взаимодействия с пользователями
grouped_df[grouped_df['vacancy_id_'] == 210068].sort_values('eventStrength', ascending=False).head(10)

,cookie_id,vacancy_id_,eventStrength
4271237,e9647deb68d6489c8f23fd7ff250f18c,210068,38.0
562580,1e9223caa87c453da3a102f3020327f7,210068,22.0
3160724,ac5710432b184069b1a4c0296fe3e41a,210068,11.0
3234089,b08259ccb54a4e9b8fe4c5bf2dc0d180,210068,8.0
1243846,43de234dae2a41f98f394d8e1e615528,210068,8.0
4672681,ffac1d4fb8ad4bb3a625d05f037a7f7f,210068,8.0
4590495,fb3eebb8cfc54a3085a07e7f47dbed4c,210068,8.0
312745,110a687ec500431289aa6116974ffc36,210068,7.0
1593021,56e2163ea88245ac8a702bdc1ce4f4c5,210068,7.0
754056,2901a05b78a0471a9f5ad505ee70168d,210068,6.0


In [338]:
# посмотрим на одного пользователя в ключе его взаимодействия с вакансиями
grouped_df[grouped_df['cookie_id'] == 'ac5710432b184069b1a4c0296fe3e41a'].sort_values('eventStrength', ascending=False).head(10)

,cookie_id,vacancy_id_,eventStrength
3160729,ac5710432b184069b1a4c0296fe3e41a,221774,37.0
3160709,ac5710432b184069b1a4c0296fe3e41a,114808,25.0
3160734,ac5710432b184069b1a4c0296fe3e41a,250491,21.0
3160712,ac5710432b184069b1a4c0296fe3e41a,136830,21.0
3160723,ac5710432b184069b1a4c0296fe3e41a,195789,20.0
3160716,ac5710432b184069b1a4c0296fe3e41a,168822,20.0
3160733,ac5710432b184069b1a4c0296fe3e41a,249034,19.0
3160732,ac5710432b184069b1a4c0296fe3e41a,248398,19.0
3160730,ac5710432b184069b1a4c0296fe3e41a,239208,15.0
3160724,ac5710432b184069b1a4c0296fe3e41a,210068,11.0


Следующий блок - это категоризация cookie_id для целей формирования разряженной матрицы (матрица принимает только числовые значения)

---

In [339]:
grouped_df['cookie_id'] = grouped_df['cookie_id'].astype("category") # наименование вакансии в категорию
grouped_df['cookie_id_num'] = grouped_df['cookie_id'].cat.codes # вот тут выделяем числовые значения

Продолжаем готовить данные для разряженных матриц

---

In [340]:
grouped_df = grouped_df[['cookie_id', 'cookie_id_num', 'vacancy_id_', 'eventStrength']]

In [341]:
# создадим датафрейм, на базе которого будет создаваться разряженная матрица
matrix_data = grouped_df[['cookie_id_num', 'vacancy_id_', 'eventStrength']]

Вот тут был блок нормализации данных, но нормализация по среднему дает аналогичный резултат что и без нее, а нормализация min max сущесвенно ухудшает метрику, поэтому от нормализации мы отказались

---

In [342]:
matrix_data.head() # мне кажется минусы нам не нужны будут проверю другую
# Вот этот датафрейм пойдет в матрицы (в следующий прогон загоним с нормализацией)

,cookie_id_num,vacancy_id_,eventStrength
0,0,137659,2.0
1,0,153975,2.0
2,0,174953,3.0
3,0,176171,2.0
4,0,182445,4.0


In [343]:
# а вот это остается нашим базовым информационным датафреймом
grouped_df.head(10)

,cookie_id,cookie_id_num,vacancy_id_,eventStrength
0,0000c4548c3944c08972bbdc1fa4eb85,0,137659,2.0
1,0000c4548c3944c08972bbdc1fa4eb85,0,153975,2.0
2,0000c4548c3944c08972bbdc1fa4eb85,0,174953,3.0
3,0000c4548c3944c08972bbdc1fa4eb85,0,176171,2.0
4,0000c4548c3944c08972bbdc1fa4eb85,0,182445,4.0
5,0000c4548c3944c08972bbdc1fa4eb85,0,187529,2.0
6,0000d7508334414ca792c5ff66eb8c14,1,106676,2.0
7,0000d7508334414ca792c5ff66eb8c14,1,108690,2.0
8,0000d7508334414ca792c5ff66eb8c14,1,115744,2.0
9,0000d7508334414ca792c5ff66eb8c14,1,169615,2.0


Вот в этом блоке мы должны соотнести наш цифровой номер cookie со строчным, для вывода ответов

---

работаем на словарях

In [344]:
# кодируем числами все наши cookie_id
codes = grouped_df.cookie_id.astype('category')
# создаем словарик в котором ключ это код а значение текстовое поле cookie_id
cookie_dic = dict(enumerate(codes.cat.categories))

In [345]:
# проверим мы его видели чуть выше
print (cookie_dic[1])

0000d7508334414ca792c5ff66eb8c14


In [346]:
# и создадим обратный словарик
cookie_to_code = dict(zip(cookie_dic.values(), cookie_dic.keys()))

In [347]:
# проверим корректрость раскодировки - все отлично
print(cookie_to_code['0000d7508334414ca792c5ff66eb8c14'])

1


Создание разряженной матрицы

---

In [348]:
# user/item spare matrix
sparse_user_item = sparse.csr_matrix((matrix_data['eventStrength'].astype(float), (matrix_data['cookie_id_num'], matrix_data['vacancy_id_'])))

In [349]:
# проверим размерности
sparse_user_item.shape
# стоит обратить внимание на то что размерность вакансий 260 168 - это потому что первых 100000 вакансий нет
# если бы мы кодировали вкансии так же как устройства - то размерность бы была 160 168 что то такое

(330180, 260168)

In [350]:
# ну и посмотрим что там внутри
print(sparse_user_item[26])

  (0, 104434)	8.0
  (0, 127795)	8.0
  (0, 147679)	4.0
  (0, 147697)	2.0
  (0, 159468)	2.0
  (0, 250423)	2.0
  (0, 258217)	2.0
  (0, 258758)	2.0
  (0, 258856)	2.0


# Обучение модели

---

---
На данном исследовательском этапе, мы подбирали значения гиперпараметров:
 - factors = 50,
 - regularization = 0.1,
 - iterations = 50

---

In [351]:
# Объявляем модель
model_user = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)

---
Следующий исследовательский блок был связан с параметром альфа - он закомментирован в коде ниже. Этот парметр настраивает(как правило усиливает) отличия в силе взаимодействия. В нашей ситуации идеально подошло значение 2

---

Обучаем модель пользователь - вакансия

In [372]:
# Настоим параметр альфа
alpha = 2
user_item_data = (sparse_user_item * alpha).astype('double')

# обучаем модель на нашей матрице
model_user.fit(user_item_data)

  0%|          | 0/50 [00:00<?, ?it/s]

# Переходим к предсказаниям по нашей задаче

---

Загружаем данные для предсказания

In [377]:
# Это набор номеров устройств для которых мы должны следать предсказания
test = pd.read_parquet('/content/test_private_users_mfti.parquet', engine='pyarrow')

In [378]:
test.head()

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a


In [379]:
# получаем для них столбец с номерными вариантами cookie как мы это закодировали ранее
test['cookie_id_num'] = test['cookie_id'].map(cookie_to_code)

In [380]:
test.head()

,cookie_id,cookie_id_num
0,0018914ba3e54011b28fa715583d3354,109
1,0035c298d8c64f368ae730a9cca9bb20,275
2,00956458877448ec9fba87fb97443fdf,722
3,0099387c921b41e7bae6c99dd8254b60,739
4,009f65e8ae99413a8da94a491320580a,766


In [381]:
# проверим перевод в коды на правильность
print(cookie_to_code['0018914ba3e54011b28fa715583d3354'])
print(cookie_to_code['0035c298d8c64f368ae730a9cca9bb20'])
print(cookie_to_code['0099387c921b41e7bae6c99dd8254b60'])

109
275
739


In [382]:
# получаем из датафрейма список пользователей, которых мы загрузим в модель
test_list = list(test['cookie_id_num'])
test_list_str = list(test['cookie_id'])
print(test_list[:5])
print(test_list_str[:5])

[109, 275, 722, 739, 766]
['0018914ba3e54011b28fa715583d3354', '0035c298d8c64f368ae730a9cca9bb20', '00956458877448ec9fba87fb97443fdf', '0099387c921b41e7bae6c99dd8254b60', '009f65e8ae99413a8da94a491320580a']


Вот в тут происходит получение предсказания и его вывод

---

In [383]:
# Ячейка поличения предсказаний, загружаем в модель полученный список кодов пользоватлей 
userids = test_list
ids, scores = model_user.recommend(userids, user_item_data[userids], N=5, filter_already_liked_items=True)

In [388]:
# а вот тут мы делаем хитро в первую колонку записываем строковые имена пользователей, а в список рекомендаций полученные предсказания
test_prediction = pd.DataFrame({'cookie_id' : test_list_str, 'predictions' : list(ids)})

In [389]:
# посмотрим что получилось (выглядит красиво как и требуется)
test_prediction.head()

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[253946, 208761, 182382, 253817, 137702]"
1,0035c298d8c64f368ae730a9cca9bb20,"[135430, 138123, 254514, 109360, 260154]"
2,00956458877448ec9fba87fb97443fdf,"[260154, 253678, 116900, 246285, 171332]"
3,0099387c921b41e7bae6c99dd8254b60,"[173337, 207423, 197407, 204473, 167761]"
4,009f65e8ae99413a8da94a491320580a,"[138123, 120188, 150516, 148686, 193142]"


In [386]:
# Сверим то что мы полчили с образцом 
test_sample = pd.read_parquet('/content/test_private_sample_submission_mfti.parquet', engine='pyarrow')

In [387]:
# один в один
test_sample.head()

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"


In [391]:
# запишем наш результат в файл
test_prediction.to_parquet('our_test_private_sample_submission_mfti.parquet')

In [393]:
# проверим файл с ответом, который мы сохранили
answer = pd.read_parquet('/content/our_test_private_sample_submission_mfti.parquet', engine='pyarrow')

In [394]:
answer.head()

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[253946, 208761, 182382, 253817, 137702]"
1,0035c298d8c64f368ae730a9cca9bb20,"[135430, 138123, 254514, 109360, 260154]"
2,00956458877448ec9fba87fb97443fdf,"[260154, 253678, 116900, 246285, 171332]"
3,0099387c921b41e7bae6c99dd8254b60,"[173337, 207423, 197407, 204473, 167761]"
4,009f65e8ae99413a8da94a491320580a,"[138123, 120188, 150516, 148686, 193142]"


## Ранжируем элементы для пользователя

---

Вот еще одна функция "из коробки" библиотеки implicit, вариант использования которой в продукте мы хотим предложить.  

Функция: rank_items()

Ранжирует заданные элементы для пользователя и возвращает отсортированный список элементов.

Параметры:

- userid (int) – идентификатор пользователя для вычисления рекомендаций
- user_items (csr_matrix) – разреженная матрица формы (number_users, number_items). Это позволяет нам (необязательно) пересчитывать пользовательские факторы (см. параметр reconculate_user) по мере необходимости
- selected_items (List of itemids) – (список идентификаторов элементов)
- recalculate_user (bool, необязательно) – При значении true не полагайтесь на сохраненное состояние пользователя и вместо этого пересчитывайте из переданных в user_items



Возвращаемый тип:	
Список

---
Допустим у нас есть определенных вакансий показ которых мы должны реализовать с учетом рекламных соглашений, мы модем применить этот список к конкретному пользователю (списку пользователей) и рекомендовать ему(им) вакансии в ранжированной последовательности по рассчитанному скорингу, что существенно увеличит вероятность положительного отклика в всеобщему удовлетворению

---

In [392]:
userid = 100005 # возьмем допустим пользователя под номером 100005
# и посмотрим в каком порядке ему лучше рекомендовать вакансии вошедшие в рекламный пул (скажем недели)
rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004, 200004, 200105, 187001, 120408])
# и получаем ответ
print(rank_elements[0]) 

[100002 187001 200105 100001 100003 200004 100004 120408]


<ipython-input-392-c695e0c8cc90>:3: DeprecationWarning: rank_items is deprecated. Use recommend with the 'items' parameter instead
  rank_elements = model_user.rank_items(userid, user_item_data, selected_items = [100001, 100002, 100003, 100004, 200004, 200105, 187001, 120408])
